# Generating Metadata and Linking Similar Dataset

Combining everything and saving files with data which can be used by D3.js to create network diagrams.

In [ ]:
#| default_exp utils.generate_Metadata

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import database_compendium.utils.ONS_scraper_functions as osf
import database_compendium.utils.Nomis_scraper_functions as nsf
import database_compendium.utils.insolvency_stats_scrapers as iss
import database_compendium.utils.NHS_QualityOutcomes_scrapers as nqos
import database_compendium.utils.police_data_scrapers as pds

import numpy as np
import requests
import pandas as pd
import math

In [ ]:
#| export
def createMetadata():

    """ ONS Data """
    
    print("Loading ONS...")
    
    # Takes over 70 mins to run
    titles, descriptions = osf.get_ONS_datasets_titles_descriptions()
    urls = osf.get_ONS_datasets_urls()
    long_desc = osf.get_ONS_long_description()
    
    latest_release = []
    cols = []
    col_data = []
    count = 0
    
    for url in urls:
        response = requests.get(url)
        try: 
            latest_release.append(response.json()['release_date'])
        except:
            latest_release.append(None)
    
        try:
            cols.append(osf.find_ONS_cols(url))
        except:
            cols.append('')
    
    
        try:
            col_data.append(osf.find_ONS_cols_and_unique_vals(url))
        except:
            col_data.append('')
    
        
        count +=1
    
    ONS_df = pd.DataFrame({'Title': titles, 'Description': descriptions, 
                           'Long_description': long_desc, 'Columns': cols, 
                           'Unique_parameters': col_data, 'Latest_release': latest_release})
    
    
    """ Nomis Data """
    
    print("Loading Nomis...")
    
    # Takes around 50 mins
    titles, descriptions, l_descriptions = nsf.get_nomis_datasets_titles_descriptions()
    latest_release = nsf.get_nomis_last_updated()
    cols = np.empty(len(titles))
    params = nsf.get_nomis_datasets_parameters()
    
    nomis_df = pd.DataFrame({'Title': titles, 'Description': descriptions, 
                             'Long_description': l_descriptions, 'Columns': cols, 
                             'Unique_parameters': params, 'Latest_release': latest_release})


    """ Monthly Insolvency Statistics """
    
    print("Loading insolvency stats")
    cols = []
    col_data = []
    insolvency_stats, long_desc = iss.get_insolvency_stats()
    titles = insolvency_stats.keys()
    
    # The descriptions on latest releases are all the same so making a repeated list
    description = [iss.get_mis_description()] * len(titles)
    latest_release = [iss.get_mis_last_updated()] * len(titles)
    long_desc = [long_desc] * len(titles)
    
    for title in titles:
        cols.append(list(insolvency_stats[title].columns))
        col_data.append(iss.get_insolvency_unique_column_vals(insolvency_stats[title]))
    
    insolvency_df = pd.DataFrame({'Title': titles, 'Description': description, 
                              'Long_description': long_desc, 'Columns': cols, 
                              'Unique_parameters': col_data, 'Latest_release': latest_release})

    
    """ Police Data - currently only for Bethnal green so is commented out """
    
    print("Loding police data")
    url = 'https://nihr.opendatasoft.com/api/records/1.0/search/?dataset=westminster-parliamentary-constituencies&rows=650'
    response = requests.get(url)
    records = response.json()['records']

    constituency_coords = {}

    for record in records:
        coords = []
        name = record['fields']['pcon22nm'] # This will change as the dataset gets updated each year
        temp_coords = record['fields']['geo_shape']['coordinates']
    
        temp_coords = pds.flatten_list(temp_coords)
    
        offset = math.floor(len(temp_coords)/4)
        constituency_coords[name] = (str(round(temp_coords[0][1], 3)) + ',' + str(round(temp_coords[0][0], 3)) + ':')
        constituency_coords[name] += (str(round(temp_coords[offset][1], 3)) + ',' + str(round(temp_coords[offset][0], 3)) + ':')
        constituency_coords[name] += (str(round(temp_coords[offset*2][1], 3)) + ',' + str(round(temp_coords[offset*2][0], 3)) + ':')
        constituency_coords[name] += (str(round(temp_coords[offset*3][1], 3)) + ',' + str(round(temp_coords[offset*3][0], 3)))

    street_level_crimes, sl_last_updated = pds.get_street_level_crimes(constituency_coords['Bethnal Green and Bow'], '2023-03', 'poly')
    no_loc_crimes = pds.get_crimes_no_loc('metropolitan', '2023-03')
    stop_searches, ss_last_updated = pds.get_stop_searches(constituency_coords['Bethnal Green and Bow'], '2023-03', 'poly')
    searches_no_loc = pds.get_searches_no_loc('metropolitan', '2023-03')

    col_data = []
    col_data.append(pds.get_unique_col_vals(street_level_crimes))
    col_data.append(pds.get_unique_col_vals(no_loc_crimes))
    col_data.append(pds.get_unique_col_vals(stop_searches))
    col_data.append(pds.get_unique_col_vals(searches_no_loc))
    
    street_level_crimes = pd.json_normalize(street_level_crimes)
    street_level_crimes = street_level_crimes.drop(columns='persistent_id')
    
    no_loc_crimes = pd.json_normalize(no_loc_crimes)
    no_loc_crimes = no_loc_crimes.drop(columns='persistent_id')
    
    stop_searches = pd.json_normalize(stop_searches)
    
    searches_no_loc = pd.json_normalize(searches_no_loc)
    searches_no_loc = searches_no_loc.drop(columns='datetime')

    titles = ['Police Data - Street-level crimes', 'Police Data - Crimes with no location',
          'Police Data - Stop and searches by area', 'Police Data - Stop and searches with no location']
    descriptions = ['Crimes at street-level; either within a 1 mile radius of a single point, or within a custom area.',
                    'Returns a list of crimes that could not be mapped to a location.',
                    'Stop and searches at street-level; either within a 1 mile radius of a single point, or within a custom area.',
                    'Stop and searches that could not be mapped to a location.']
    latest_release = [sl_last_updated, sl_last_updated, ss_last_updated, ss_last_updated]
    
    f = open('data/police_long_descriptions.txt', 'r')
    long_desc = f.read().split('\n')
    
    cols = []
    for data in col_data:
        cols.append(list(data.keys()))

    police_df = pd.DataFrame({'Title': titles, 'Description': descriptions, 
                          'Long_description': long_desc, 'Columns': cols, 
                          'Unique_parameters': col_data, 'Latest_release': latest_release})

    metadata_df = pd.concat([ONS_df, nomis_df, insolvency_df, police_df])
    print("Complete")

    return metadata_df

In [ ]:
metadata_df = createMetadata()

Loading ONS...
337
337
337
337
337
337
Loading Nomis...
Loading insolvency stats
Loding police data
Complete


In [ ]:
metadata_df

,Title,Description,Long_description,Columns,Unique_parameters,Latest_release
0,Quarterly personal well-being estimates,Seasonally and non seasonally-adjusted quarter...,We are currently reviewing the measures of nat...,"[v4_2, LCL, UCL, yyyy-qq, Time, uk-only, Geogr...",,2023-05-12T00:00:00.000Z
1,Personal well-being estimates by local authority,"Estimates of life satisfaction, feeling that t...",We are currently reviewing the measures of nat...,"[v4_3, Data marking, Lower limit, Upper limit,...",,2022-10-31T00:00:00.000Z
2,Deaths registered weekly in England and Wales ...,Provisional counts of the number of deaths reg...,Quality and methodology information for mortal...,"[v4_1, Data Marking, calendar-years, Time, adm...",,2023-08-01T00:00:00.000Z
3,Death registrations and occurrences by local a...,Provisional counts of the number of deaths reg...,Quality and methodology information for mortal...,"[v4_0, calendar-years, Time, administrative-ge...",,2023-08-03T00:00:00.000Z
4,Death registrations and occurrences by health ...,Provisional counts of the number of deaths reg...,Quality and methodology information for mortal...,"[v4_0, calendar-years, Time, local-health-boar...",,2023-08-03T00:00:00.000Z
...,...,...,...,...,...,...
12,"Table 11: Individual insolvencies, Northern Ir...",Monthly company and individual insolvency stat...,1 January 2019 to 30 June 2023\nThe Insolvency...,"[Month, Total individual insolvencies, Bankrup...","{'Month': ['Jan 19', 'Feb 19', 'Mar 19', 'Apr ...",18 July 2023
0,Police Data - Street-level crimes,Crimes at street-level; either within a 1 mile...,The Police API Documentation for Street-Level ...,"[category, location_type, context, location_su...","{'category': ['anti-social-behaviour', 'bicycl...",2023-06-01
1,Police Data - Crimes with no location,Returns a list of crimes that could not be map...,The Police API Documentation for Crimes with N...,"[category, context, location_subtype, outcome_...","{'category': ['bicycle-theft', 'burglary', 'cr...",2023-06-01
2,Police Data - Stop and searches by area,Stop and searches at street-level; either with...,The Police API Documentation for Stop and Sear...,"[outcome, self_defined_ethnicity, gender, legi...","{'outcome': ['A no further action disposal', '...",2023-06


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()